In [9]:
# Required dependencies
# 1. NLTK
# 2. Gensim for word2vec
# 3. Keras with tensorflow/theano backend

#=======================
import json
import codecs
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import keras
import keras.backend as K
from keras.layers import Reshape, Concatenate, Activation, LeakyReLU, Lambda, Bidirectional
from gensim.test.utils import common_texts, get_tmpfile #10-11
import gc
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords 
from nltk.tokenize import TweetTokenizer
from math import exp, log, e
import tensorflow as tf
from keras import metrics
from IPython.display import Image
import time
import math
import matplotlib.pyplot as plt
from keras.layers.normalization import BatchNormalization
#========================

import numpy as np
np.random.seed(1337)
import json, re, nltk, string
from nltk.corpus import wordnet
from gensim.models import Word2Vec
from keras.preprocessing import sequence
from keras.models import Model, model_from_json
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, merge, Masking, TimeDistributed
from keras.optimizers import RMSprop, Adam
from keras.utils import np_utils, plot_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, confusion_matrix, mean_squared_error

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
eclipse = '/data/projects/eclipse.json'
chrome = '/data/projects/chromium.json'
firefox = '/data/prjects/firefox.json'

In [3]:
#========================================================================================
# Initializing Hyper parameter
#========================================================================================
#1. Word2vec parameters
min_word_frequency_word2vec = 30
embed_size_word2vec = 200
context_window_word2vec = 5
combined_word2vec = True

#2. Classifier hyperparameters
numCV = 10
max_sentence_len = 150
min_sentence_length = 15
max_his_len = 10
max_activity_len = 10
min_activity_len = 4
batch_size = 32
label_num=100
class_std = 3 # 1=52%

In [14]:
#==============================================================================
# Preprocess 3 (ACTIVITY-BASED -> DATE-BASED)
#==============================================================================
path = eclipse
data_name = {eclipse:'eclipse', chrome:'chrome', firefox:'firefox'}

with_sys=False

word2vec_train = True
if word2vec_train:
    all_data_word2vec = []
accumulate =False
cut_activity_len = True
max_activity_len = 10
noise_day = 3 
limit_day = 100
recentday = 3 # all_recentday

stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'had', 'having', 'doing', 'a', 'an', 'the', 'and', 'or', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into', 'through', 'during', 'before', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'too', 'very', 's', 't', 'd', 'll', 'm', 'o', 're', 've', 'y',  'ma']
stop_words += list(string.ascii_lowercase)
stop_words = set(stop_words)

with open(path, encoding='utf-8') as data_file:
    data = json.loads(data_file.read(), strict=False)
    
all_data = []
all_history = []
all_time = []
#all_stream = []
all_bugid = []
all_total =[]

all_workday = []
all_recentday = []
all_openday = []
all_activitycnt = []
all_cc = []
all_writer = []
all_commcnt = []
all_hiscnt = []
all_wordcnt = []

start=0
for item in data:
    # 'except' condition
    if path==eclipse and item['activity']!=[] and int(item['activity'][0][1].split('-')[0])<2013:
        continue
    if len(all_time)>=100000: break
    if len(item['days']) == 0 or item['days'][0] >= limit_day or item['days'][-1]>noise_day:
        continue
        
    # title
    current_title = item['title'].replace('\r', ' ')
    current_title= re.sub(r'(\w+)0x\w+', '', current_title)
    current_title = current_title.lower()
    current_title = re.sub(r"[!#$%&'()*+,./:;<=>?@\^_`{|}~-]", ' ', current_title)
    current_title_tokens = nltk.word_tokenize(current_title)
    current_title_filter = [word.strip(string.punctuation) for word in current_title_tokens]
    
    unique_date = [e[1].split(" ")[0] for e in item['activity']]
    unique_date = sorted(list(set(unique_date)))
    if cut_activity_len and len(unique_date)>max_activity_len:
        continue
    
    curr_activityCnt = 0
    curr_workday = 0
    #curr_stream = [0]*limit_day

    curr_data_list = []
    curr_history_list = []
    for j,date in enumerate(unique_date):
        curr_desc = " "
        curr_his = " "
        curr_workday +=1
        this_activityCnt = 0
        total_act = 0
        comm_cnt = 0
        his_cnt = 0 
        tmp_cc = []
        for idx, x in enumerate(item['activity']):
            if x[1].split(' ')[0] > date:
                break   
            if not accumulate:
                if x[1].split(' ')[0] < date:
                    continue  
            if x[2]!="":
                total_act+=1
                comm_cnt +=1
            if x[3]!="":
                total_act+=1
                his_cnt +=1
          
            aft_x0 = re.sub(r'\b(Comment)(\s+)(\d+)', r'\1\3', x[0])
            if aft_x0.split()[0]=='Reported' or 'Description' in x[0]:
                if aft_x0.split()[0]=='Reported':
                    reporter = aft_x0.split()[2]
                else:
                    reporter = x[0].split()[0]
    
            if path==firefox:
                cc = x[0].split()[0].strip(', ')
            elif path ==chrome:
                cc = aft_x0.split()[2].strip()
            elif path==eclipse:
                cc= x[0].split()[0]
            tmp_cc.append(cc)
            curr_desc += aft_x0+' '+x[2]+' '      
            if with_sys:
                aft_x3 = x[3].replace(':',' : ').replace('>', ' > ')
                current_desc += aft_x3 + ' '
            curr_his += x[3].replace(':',' : ').replace('>', ' > ') + ' '
            curr_time = item['days'][idx]
            curr_commentDate = item['commentDate'][idx]
            curr_activityCnt+=1
            this_activityCnt+=1
        curr_active = [0]*recentday
        for idx, x in enumerate(item['activity']):
            for p in range(recentday,0,-1):
                if item['days'][idx]-curr_time==p:
                    curr_active[-p] = 1
        
        curr_desc = curr_desc.split()
        curr_desc = [x for x in curr_desc if len(x)<100]
        curr_desc = " ".join(curr_desc)
        curr_desc = curr_desc.replace('\r', ' ')
        curr_desc = re.sub(r"(\w+[\w\.]*)@(\w+[\w\.]*)\.([A-Za-z]+)", r'\1', curr_desc)
        curr_desc = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', curr_desc)
        curr_desc = re.sub(r'(\w*)0x\w+', ' ', curr_desc)
        #curr_desc = re.sub(r'[^(A-Za-z0-9\s)]+', ' ', curr_desc) # Solaris-specific >> solaris, specific
        #print(5,curr_desc)
        curr_desc = re.sub(r"(\s+)\d+", " ", curr_desc)
        curr_desc = curr_desc.lower()
        curr_desc_tokens = nltk.word_tokenize(curr_desc) #word_tokenize
        curr_desc_filter = [word.strip(string.punctuation) for word in curr_desc_tokens]
        curr_data = (current_title_filter + curr_desc_filter) if j==0 else curr_desc_filter
        curr_data = list(filter(None, curr_data))
        curr_data = [w for w in curr_data if not w in stop_words]
        
        curr_his = curr_his.replace('✿','')
        curr_his = re.sub(r"(\w+[\w\.]*)@(\w+[\w\.]*)\.([A-Za-z]+)", r'\1', curr_his) # mail-address
        curr_his = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', curr_his)
        #curr_his = re.sub(r'Attachment(\s+)[#][0-9]+(\s+)-', '', curr_his)
        curr_his = curr_his.lower()
        curr_his_tokens = nltk.word_tokenize(curr_his)
        curr_his_filter = [word.strip(string.punctuation) for word in curr_his_tokens]
        curr_history = list(filter(None, curr_his_filter))
        curr_history = [w for w in curr_history if not w in stop_words]
        
        if word2vec_train and (curr_data+curr_history)!=[]:
            all_data_word2vec.append(curr_data+curr_history)

        #curr_stream[curr_commentDate] = this_activityCnt
        #for x in range(curr_commentDate):
        #    if curr_stream[x]== 0:
        #        curr_stream[x] = -1
        curr_data_list.append(curr_data)
        curr_history_list.append(curr_history)
        all_time.append(curr_time)
        #all_stream.append(list(curr_stream))
        all_bugid.append(item['reportNum'])
        all_total.append(item['days'][0])

        all_commcnt.append(comm_cnt)
        all_hiscnt.append(his_cnt)
        all_wordcnt.append(len(curr_history)+len(curr_data))
        all_workday.append(curr_workday)
        all_recentday.append(curr_active)
        all_openday.append(curr_commentDate)
        all_activitycnt.append(total_act)
        all_cc.append(len(set(tmp_cc)))

        min_writer = 0
        if path == firefox:
            tmp_writer = [0,0,0]
            if 'reporter' in curr_data:
                tmp_writer[0]=1
                min_writer+=1
            if 'assignee' in curr_data:
                tmp_writer[1]=1
                min_writer+=1
            if len(tmp_cc)>min_writer:
                tmp_writer[2]=1
        elif path==chrome or path ==eclipse:
            tmp_writer = [0,0]
            if reporter in tmp_cc:
                tmp_writer[0]=1
                min_writer+=1
            if len(tmp_cc)>min_writer:
                tmp_writer[1]=1
        all_writer.append(tmp_writer)

        if start==0:
            start = item['activity'][0][1]
            print('start', start)
        end = item['activity'][0][1]

    all_data.append(curr_data_list)
    all_history.append(curr_history_list)

del data
gc.collect()
print(len(all_data))
print(len(all_time))
print('end', end)

start 2014-03-06 19:00:33
30428
100001
end 2015-11-05 19:54:33


In [15]:
# Load Word2vec

if combined_word2vec:
    if path == chrome:
        wordvec_path = '/data/word2vec/chrome_combined200.model'
    elif path ==eclipse:
        wordvec_path = '/data/word2vec/eclipse_combined200.model'
    elif path ==firefox:
        wordvec_path = '/data/word2vec/firefox_combined200.model'
    wordvec_model = Word2Vec.load(wordvec_path)
    vocabulary = wordvec_model.wv.vocab
else: 
    wordvec_path_1 = '/data/word2vec/newProject_all_data_word2vec.model'
    wordvec_path_2 = '/data/word2vec/newProject_all_history_word2vec.model'
    wordvec_model_1 = Word2Vec.load(wordvec_path_1)
    wordvec_model_2 = Word2Vec.load(wordvec_path_2)
    vocabulary_1 = wordvec_model_1.wv.vocab
    vocabulary_2 = wordvec_model_2.wv.vocab

print(len(vocabulary))

27696


In [13]:
# seq-task implement

auto_balancing=False
train_balancing=False
data_name = {mozilla:'mozilla', chrome:'chrome', firefox:'firefox', eclipse:'eclipse'}
#[0]:workday [1]:openday [2]:activityCnt [3]:cc 
total_split = {'firefox': [[3,7],[1,2,3,10,30],[3,7,20,50],[1,2,3]], 'chrome':[[3,7],[1,2,3,10,30],[1,2,5,10],[1,2,3]], 'eclipse':[[3,7],[1,2,3,10,30],[1,2,3,5],[1,2,3]]} 

test_mode = False
if test_mode: save_off = True
else: save_off = False
save_off=True

emb_load = True

max_activity_len = 10
min_activity_len = 1 # workday 1-10
lambda_list = [7]


#Task Type
bidirectional = False
ResLSTM = True
task_mlp = False

task2_nextAct = False
task3_outlier = False
task4_lifetime = False
task5_multiFixtime = False

# data size
d_sizes = [1]

#
for d_size in d_sizes:
    print('============= d_size =============', d_size)
    for lamb in lambda_list:
        #if lamb==2 or lamb==4: continue
        emb_std = lamb
        mode_list = [0] # 0,1 모두 돈다
        if task2_nextAct:
            seq_std_list = [1,2,3]
            label_num=2
        elif task3_outlier:
            seq_std_list = [60]
            label_num=2
        elif task4_lifetime:
            seq_std_list = [100]
            label_num=100
        elif task5_multiFixtime:
            seq_std_list = [100]
            label_num=100
        else: # task_1
            seq_std_list = [5,4,3,2,1]
            label_num=2
        #
        for seq_std in seq_std_list:
            if task3_outlier:
                all_class = [1 if x>=seq_std else 0 for x in all_total]
            else:
                all_class = [0 if x<=seq_std else 1 for x in all_time]
            c0 = all_class.count(0)
            c1 = all_class.count(1)
            class_percent = c1/(c0+c1)
            print('class 1 비율', class_percent*100)
            if auto_balancing:
                if class_percent<=0.4 or class_percent>=0.6:
                    train_balancing=True
                else:
                    train_balancing=False
            #
            m3_runT = []
            m5_runT = []
            for mode in mode_list: # 0:3-5, 1:4-6
                if mode==1: continue
                if mode==0:
                    doc2vec_rnn = True
                    m3_stacking = False
                    doc2vec_avg = False
                    multi_layer_LSTM = False
                elif mode ==1:
                    doc2vec_rnn = False 
                    m3_stacking = False
                    doc2vec_avg = True
                    multi_layer_LSTM = False
                for i in range(6,11):
                    totalLength = len(all_data)
                    print('Total length: ', totalLength)
                    splitLength = int(totalLength / (numCV + 1))
                    print('emb_std:',emb_std)
                    print('seq_std:', seq_std)
                    print('train_balancing', train_balancing)
                    # Split cross validation set
                    print ('CV',i)
                    if test_mode:
                        train_data = all_data[:30]#i*splitLength]
                        train_history = all_history[:30]#i*splitLength]
                        train_3to1 = sum([len(r) for r in train_data])       
                        train_time = all_time[:train_3to1]#i*splitLength]
                        train_workday = all_workday[:train_3to1]
                        train_recentday = all_recentday[:train_3to1]
                        train_openday = all_openday[:train_3to1]
                        train_activitycnt = all_activitycnt[:train_3to1]
                        train_cc = all_cc[:train_3to1]
                        train_writer = all_writer[:train_3to1]
                        train_bugid = all_bugid[:train_3to1]
                        train_total = all_total[:train_3to1]
                    else:
                        train_data = all_data[:int(i*splitLength*d_size)]
                        train_history = all_history[:int(i*splitLength*d_size)]
                        train_3to1 = sum([len(r) for r in train_data])       
                        train_time = all_time[:train_3to1]#i*splitLength]
                        train_workday = all_workday[:train_3to1]
                        train_recentday = all_recentday[:train_3to1]
                        train_openday = all_openday[:train_3to1]
                        train_activitycnt = all_activitycnt[:train_3to1]
                        train_cc = all_cc[:train_3to1]
                        train_writer = all_writer[:train_3to1]
                        train_bugid = all_bugid[:train_3to1]
                        train_total = all_total[:train_3to1]
                    # ===================================================================== 
                    updated_train_data = []    
                    updated_train_history = []    
                    updated_train_time = []
                    #updated_train_stream = []
                    #updated_train_workday = []
                    #updated_train_bugid = []

                    j=0
                    for bug1, bug2 in zip(train_data, train_history):
                        train_data_list = []
                        train_history_list = []
                        if len(bug1)>=min_activity_len:
                            for act1, act2 in zip(bug1, bug2):
                                current_train_filter = [word for word in act1 if word in vocabulary]
                                his_current_train_filter = [word for word in act2 if word in vocabulary]
                                train_data_list.append(current_train_filter)
                                train_history_list.append(his_current_train_filter)
                                #updated_train_time.append(train_time[j])
                                #updated_train_stream.append(train_stream[j])
                                #updated_train_workday.append(train_workday[j])
                                #updated_train_bugid.append(train_bugid[j])
                                j+=1
                            updated_train_data.append(train_data_list)
                            updated_train_history.append(train_history_list)
                        else:
                            j+=len(bug1)

                    del train_data, train_history, #train_bugid, train_workday
                    gc.collect()

                    # ===================================================================== 
                    updated_train_time = [1 if x<=seq_std else 0 for x in train_time]
                    curr_split = total_split[data_name[path]][0]
                    updated_train_workday = [0 if x<=curr_split[0] else 1 if x<=curr_split[1] else 2 for x in train_workday]
                    updated_train_recentday = []
                    for x in train_recentday:
                        tmp = 0 
                        for j,y in enumerate(x):
                            tmp+= pow(2,j)*y
                        updated_train_recentday.append(tmp) 
                    curr_split = total_split[data_name[path]][1]
                    updated_train_openday = [0 if x<=curr_split[0] else 1 if x<=curr_split[1] else 2 if x<=curr_split[2] else 3 if x<=curr_split[3] else 4 for x in train_openday]
                    curr_split = total_split[data_name[path]][2]
                    updated_train_activitycnt = [0 if x<=curr_split[0] else 1 if x<=curr_split[1] else 2 if x<=curr_split[2] else 3 if x<=curr_split[3] else 4 for x in train_activitycnt]
                    curr_split = total_split[data_name[path]][3]
                    updated_train_cc = [0 if x<=curr_split[0] else 1 if x<=curr_split[1] else 2 if x<=curr_split[2] else 3 for x in train_cc]
                    updated_train_writer = train_writer
                    updated_train_bugid = train_bugid
                    updated_train_data_1d = [y for x in updated_train_data for y in x]
                    updated_train_history_1d = [y for x in updated_train_history for y in x]
                    train_col_size = [len(x) for x in updated_train_data]
                    train_n = len(updated_train_data_1d)
                    train_df =pd.DataFrame({'x1': updated_train_data_1d, 'x2': updated_train_history_1d, 
                                            's1': updated_train_workday, 's2':updated_train_recentday, 's3':updated_train_activitycnt,
                                            's4':updated_train_cc, 's5':updated_train_writer, 's6':updated_train_openday,
                                            'bugid':updated_train_bugid,
                                            'y1': train_time, 'y2':updated_train_time})
                    #print(train_df[:10])
                    del updated_train_time, updated_train_history, updated_train_data, updated_train_data_1d, updated_train_history_1d
                    del updated_train_workday, updated_train_recentday, updated_train_activitycnt, updated_train_cc, updated_train_writer, updated_train_bugid
                    gc.collect()

                    # Task type
                    if task2_nextAct:
                        train_nextact = []
                        q=0
                        for col_size in train_col_size:
                            for x in range(col_size):
                                if x==col_size-1:
                                    train_nextact.append(-1)
                                else:
                                    train_nextact.append(train_time[q+x]-train_time[q+x+1])
                            q+=col_size
                        train_df['y1'] = train_nextact
                        train_df = train_df[train_df.y1!=-1]
                        train_n = len(train_df)
                        # new train_col_size
                        cnt=0
                        train_col_size = []
                        prev_id = train_df.iloc[0]['bugid']
                        for x in train_df.bugid.values:
                            if prev_id != x:
                                train_col_size.append(cnt)
                                cnt = 0
                            prev_id = x
                            cnt+=1
                        train_col_size.append(cnt)
                    elif task3_outlier or task4_lifetime:
                        train_df['y1'] = train_total
                    
                    # =============================
                    # BALANCE RESAMPLING
                    if train_balancing:
                        class_count = train_df.y2.value_counts()
                        print('\nClass 0:', class_count[0])
                        print('Class 1:', class_count[1])
                        df_class_0 = train_df[train_df['y2']==0]
                        df_class_1 = train_df[train_df['y2']==1]
                        if class_count[0]<class_count[1]:
                            under_class_cnt = class_count[0]
                            df_class_1 = df_class_1.sample(under_class_cnt)
                        else:
                            under_class_cnt = class_count[1]
                            df_class_0 = df_class_0.sample(under_class_cnt)
                        train_df = pd.concat([df_class_0, df_class_1], axis=0)
                        train_df = train_df.sort_index()
                        train_n = len(train_df)
                        print(train_df[:10],'\n')
                        print('\nRandom under-sampling:')
                        print(train_df.y2.value_counts(),'\n')
                        # new train_col_size
                        cnt=0
                        train_col_size = []
                        prev_id = train_df.iloc[0]['bugid']
                        for x in train_df.bugid.values:
                            if prev_id != x:
                                train_col_size.append(cnt)
                                cnt = 0
                            prev_id = x
                            cnt+=1
                        train_col_size.append(cnt)

                    # ===================================================================== 
                    X_train_last = np.zeros(shape=[train_n, max_sentence_len, embed_size_word2vec], dtype='float32') # len(updated_train_data) # train_len*2
                    X_train_history = np.zeros(shape=[train_n, max_his_len, embed_size_word2vec], dtype='float32') # len(updated_train_history)
                    Y_train = np.empty(shape=[train_n,1], dtype='int32') # len(updated_train_time)

                    j=0
                    for curr_row1, curr_row2 in zip(train_df.x1.values, train_df.x2.values):
                        if len(curr_row1)>max_sentence_len:
                            start_loc = len(curr_row1) - max_sentence_len
                        else: 
                            start_loc = 0
                        sequence_cnt = 0    
                        for item1 in curr_row1[start_loc:]:
                            if item1 in vocabulary:
                                X_train_last[j, sequence_cnt, :] = wordvec_model[item1] 
                                sequence_cnt = sequence_cnt + 1                
                                if sequence_cnt == max_sentence_len-1:
                                    break  
                        for k in range(sequence_cnt, max_sentence_len):
                            X_train_last[j, k, :] = np.zeros((1,embed_size_word2vec))   
                        if len(curr_row2)>max_his_len:
                            start_loc = len(curr_row2) - max_his_len
                        else: 
                            start_loc = 0
                        sequence_cnt = 0
                        for item2 in curr_row2[start_loc:]:
                            if item2 in vocabulary:
                                X_train_history[j, sequence_cnt, :] = wordvec_model[item2] 
                                sequence_cnt = sequence_cnt + 1                
                                if sequence_cnt == max_his_len-1:
                                        break 
                        for k in range(sequence_cnt, max_his_len):
                            X_train_history[j, k, :] = np.zeros((1,embed_size_word2vec))
                        Y_train[j] = train_df.y1.values[j]
                        j+=1
                    #y_train = np_utils.to_categorical(Y_train, label_num)

                    if doc2vec_avg:
                        X_train_doc = np.zeros(shape=[train_n, embed_size_word2vec], dtype='float32')
                        X_train_his_doc = np.zeros(shape=[train_n, embed_size_word2vec], dtype='float32')
                        for j,words in enumerate(X_train_last):
                            cnt=0
                            smt = np.asarray([0.0 for p in range(embed_size_word2vec)])
                            for word in words:
                                if not np.any(word): # All-zero element?
                                    break
                                smt+=word
                                cnt+=1
                            if not np.any(smt):
                                avg = smt
                            else:
                                avg = smt/cnt
                            X_train_doc[j] = avg
                        for j,words in enumerate(X_train_history):
                            cnt=0
                            smt = np.asarray([0.0 for p in range(embed_size_word2vec)])
                            for word in words:
                                if not np.any(word): # All-zero element?
                                    break
                                smt+=word
                                cnt+=1
                            if not np.any(smt):
                                avg = smt
                            else:
                                avg = smt/cnt
                            X_train_his_doc[j] = avg

                        X_train_avg = np.concatenate((X_train_doc, X_train_his_doc),axis=1)
                        del X_train_last, X_train_history
                        gc.collect()

                    # Statistic
                    X_train_workday = np.asarray(train_df.s1.values, dtype=np.float32)
                    class_num = len(total_split[data_name[path]][0])+1
                    X_train_workday = np_utils.to_categorical(X_train_workday, class_num)
                    X_train_openday = np.asarray(train_df.s6.values, dtype=np.float32)
                    class_num = len(total_split[data_name[path]][1])+1
                    X_train_openday = np_utils.to_categorical(X_train_openday, class_num)
                    X_train_recentday = np.asarray(train_df.s2.values, dtype=np.float32)
                    class_num = 8
                    X_train_recentday = np_utils.to_categorical(X_train_recentday, class_num)
                    X_train_activitycnt = np.asarray(train_df.s3.values, dtype=np.float32)
                    class_num = len(total_split[data_name[path]][2])+1
                    X_train_activitycnt = np_utils.to_categorical(X_train_activitycnt, class_num)
                    X_train_cc = np.asarray(train_df.s4.values, dtype=np.float32)
                    class_num = len(total_split[data_name[path]][3])+1
                    X_train_cc = np_utils.to_categorical(X_train_cc, class_num)
                    X_train_writer = np.array([np.array(x, dtype=np.float32) for x in train_df.s5.values])

                    X_train_stat = np.concatenate((X_train_workday, X_train_recentday),axis=1)
                    X_train_stat = np.concatenate((X_train_stat, X_train_openday),axis=1)
                    X_train_stat = np.concatenate((X_train_stat, X_train_activitycnt),axis=1)
                    X_train_stat = np.concatenate((X_train_stat, X_train_cc),axis=1)
                    X_train_stat = np.concatenate((X_train_stat, X_train_writer),axis=1)

                    # Model
                    if doc2vec_rnn:
                        h1=256
                        h2 = 32
                        last_input = Input(shape=(max_sentence_len, embed_size_word2vec), dtype='float32')
                        mask_1 = Masking(mask_value=0)(last_input)
                        if bidirectional:
                            merged_1 = Bidirectional(LSTM(h1, name='merged_1'))(mask_1)
                        else:
                            forwards_1 = LSTM(h1, name='forwards_1')(mask_1)
                            backwards_1 = LSTM(h1, go_backwards=True, name='backwords_1')(mask_1)         
                            merged_1 = keras.layers.concatenate([forwards_1, backwards_1], axis=-1)
                        after_dp_1= Dropout(0.5, name='after_dp_1')(merged_1)
                        tmp_output_1 = Dense(label_num, activation='softmax', name='tmp_output_1')(after_dp_1)

                        history_input = Input(shape=(max_his_len, embed_size_word2vec), dtype='float32')
                        mask_2 = Masking(mask_value=0)(history_input)
                        if bidirectional:
                            merged_2 = Bidirectional(LSTM(h2, name='merged_2'))(mask_2)
                        else:
                            forwards_2 = LSTM(h2, name='forwards_2')(mask_2)
                            backwards_2 = LSTM(h2, go_backwards=True, name='backwards_2')(mask_2)
                            merged_2 = keras.layers.concatenate([forwards_2, backwards_2], axis=-1)
                        after_dp_2 = Dropout(0.5, name='after_dp_2')(merged_2)
                        tmp_output_2 = Dense(label_num, activation='softmax', name='tmp_output_2')(after_dp_2)

                        # stat
                        stat_input = Input(shape=(len(X_train_stat[0]),), dtype='float32')

                        middle_merge = keras.layers.concatenate([after_dp_1, after_dp_2, stat_input], axis=-1)
                        middle_dense = Dense(380, name='middle_dense_1')(middle_merge) 
                        middle_dense = LeakyReLU(alpha=0.3)(middle_dense)
                        middle_dense = Dense(450, name='middle_dense_2')(middle_dense) 
                        middle_dense = LeakyReLU(alpha=0.2)(middle_dense)
                        middle_dense = Dense(200, name='middle_dense_3', activation='tanh')(middle_dense) 
                        #middle_dense = LeakyReLU(alpha=0.5)(middle_dense)
                        #last_dense = Dense(100, name='middle_dense_4')(middle_dense) 
                        #last_dense = LeakyReLU(alpha=0.1)(last_dense)
                        output = Dense(label_num, activation='softmax', name='output')(middle_dense)
                        if m3_stacking:
                            user_model = Model(input=last_input, output = tmp_output_1)
                            sys_model = Model(input=history_input, output=tmp_output_2)
                        embedding_size = 200
                        m3 = Model(input=[last_input, history_input, stat_input], output=output)    

                    elif doc2vec_avg:
                        docAvg_input = Input(shape=(len(X_train_avg[0]),), dtype='float32')
                        stat_input = Input(shape=(len(X_train_stat[0]),), dtype='float32')
                        concat_input = keras.layers.concatenate([docAvg_input, stat_input], axis=-1)
                        middle_dense = Dense(230, name='middle_dense_1')(concat_input) 
                        middle_dense = LeakyReLU(alpha=0.3)(middle_dense)
                        middle_dense = Dense(300, name='middle_dense_2')(middle_dense) 
                        middle_dense = LeakyReLU(alpha=0.2)(middle_dense)
                        middle_dense = Dense(180, name='middle_dense_3')(middle_dense) 
                        middle_dense = LeakyReLU(alpha=0.5)(middle_dense)
                        last_dense = Dense(100, name='middle_dense_4')(middle_dense) 
                        last_dense = LeakyReLU(alpha=0.1)(last_dense)
                        output = Dense(label_num, activation='softmax', name='output')(last_dense)

                        embedding_size = 180
                        m3 = Model(input=[docAvg_input, stat_input], output=output)    

                    adam = Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
                    m3.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

                    # TRAIN MODEL
                    es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')

                    print('\n\n******************* M3-RNN *******************')  
                    if emb_load:
                        pass
                    else:
                        X_train_last_sp, X_valid_last, X_train_history_sp, X_valid_history, X_train_stat_sp, X_valid_stat, y_train_sp, y_valid = train_test_split(X_train_last, X_train_history, X_train_stat, y_train, test_size=0.15, shuffle= True)
                        m3.summary()
                        start = time.time()
                        hist = m3.fit([X_train_last_sp, X_train_history_sp, X_train_stat_sp], y_train_sp, batch_size=batch_size, 
                                         validation_data = ([X_valid_last, X_valid_history, X_valid_stat], y_valid), epochs=20, callbacks=[es])
                        end = time.time()
                        m3_runT.append(int(end-start))
                        print(" ********* emb%d-M3 Learning time: %dsec ********* " %(emb_std,int(end-start)))
                        del X_train_last_sp, X_train_history_sp, X_train_stat_sp, y_train_sp
                        del X_valid_last, X_valid_history, X_valid_stat, y_valid
                        gc.collect()
                        if not save_off:
                            # Save model
                            model_json = m3.to_json()
                            model_name = data_name[path]+"_M3_limit100_class"+str(emb_std)+"_cv"+str(i)+".json"
                            weight_name = data_name[path]+"_M3_limit100_class"+str(emb_std)+"_cv"+str(i)+".h5"
                            with open(model_name,"w") as json_file :
                                json_file.write(model_json)
                                m3.save_weights(weight_name)
                            print("Saved model to disk\n\n\n")

                    print('\n\n******************* M5- seqence RNN *******************')
                    # load seq-embedding weights
                    model_1 = Model(input=[last_input, history_input, stat_input], output=middle_dense) 
                    weight_name = data_name[path]+"_M3_tanh_limit100_class"+str(emb_std)+"_cv"+str(i)+".h5"
                    model_1.load_weights(weight_name, by_name=True)                 

                    embedding_train_output = model_1.predict([X_train_last, X_train_history, X_train_stat])
                    del X_train_last, X_train_history, X_train_stat
                    gc.collect()

                    X_train_activity = np.zeros(shape=[len(train_col_size), max_activity_len, embedding_size], dtype='float32') 
                    y_train_activity = np.full((len(train_col_size), max_activity_len, label_num), -1, dtype='int32') 
                    act_idx = 0 # 200d_activity_output row

                    for j, col_size in enumerate(train_col_size):
                        for p in range(col_size):
                            X_train_activity[j,p,:] = embedding_train_output[act_idx+p]
                            if task3_outlier:
                                if train_df.y1.values[act_idx+p]>=seq_std:
                                    y_train_activity[j,p,0] = 0
                                    y_train_activity[j,p,1] = 1
                                else:
                                    y_train_activity[j,p,0] = 1
                                    y_train_activity[j,p,1] = 0
                            elif task4_lifetime or task5_multiFixtime:
                                y_train_activity[j,p,:] = np.zeros(label_num, dtype='int32')
                                y_train_activity[j,p,train_df.y1.values[act_idx+p]] = 1
                            else: # Task1
                                if train_df.y1.values[act_idx+p]<=seq_std:
                                    y_train_activity[j,p,0] = 0
                                    y_train_activity[j,p,1] = 1
                                else:
                                    y_train_activity[j,p,0] = 1
                                    y_train_activity[j,p,1] = 0
                        act_idx += col_size
                    del embedding_train_output 
                    gc.collect()

                    # construct seq-emb model
                    if ResLSTM:
                        data_input = Input(shape=(max_activity_len, embedding_size), dtype='float32')
                        output = make_residual_lstm_layers(data_input, rnn_width=embedding_size, rnn_depth=8, rnn_dropout=0.2)
                        #task_output = TimeDistributed(Dense(label_num, activation='softmax'))(output) 
                        seq_emb = Model(input=data_input, output=output)
                    else:
                        activity_input = Input(shape=(max_activity_len, embedding_size), dtype='float32')
                        mask = Masking(mask_value=0)(activity_input)
                        forwards_1 = LSTM(256, name='forwards_1', return_sequences=True)(mask)
                        forwards_1 = Dropout(0.5)(forwards_1)
                        #forwards = LSTM(128, return_sequences=True)(forwards)
                        #forwards = Dropout(0.3)(forwards)
                        emb_output = TimeDistributed(Dense(label_num, activation='relu'))(forwards_1)
                        forwards_2 = LSTM(256, name='forwards_2', return_sequences=True)(emb_output)
                        forwards_3 = LSTM(128, name='forwards_3', return_sequences=True)(forwards_2)
                        output = TimeDistributed(Dense(2, activation='softmax'))(forwards_3)
                        seq_emb = Model(input=activity_input, output=output)  

                    adam = Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
                    #seq_emb.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
                    #seq_emb.summary()

                    # load seq-embedding model
                    model_json = data_name[path]+"_newM5_limit100_emb"+str(lamb)+"_seq"+str(lamb)+"_cv"+str(i)+".json"
                    model_h5 = data_name[path]+"_newM5_limit100_emb"+str(lamb)+"_seq"+str(lamb)+"_cv"+str(i)+".h5"
                    json_file = open(model_json, "r")
                    loaded_model_json=json_file.read()
                    json_file.close()
                    loaded_seq_emb=model_from_json(loaded_model_json)
                    loaded_seq_emb.load_weights(model_h5)

                    # get wieghts
                    weights_list = loaded_seq_emb.get_weights()
                    j=0
                    for layer1, layer2 in zip(loaded_seq_emb.layers, seq_emb.layers):
                        seq_emb.layers[j].set_weights(layer1.get_weights())
                        j+=1

                    # construct task model
                    if task_mlp:
                        activity_input = Input(shape=(max_activity_len, embedding_size), dtype='float32')
                        mask = Masking(mask_value=0)(activity_input)
                        mid_dense = TimeDistributed(Dense(120, name='mid_dense_1'))(mask) 
                        mid_dense = TimeDistributed(LeakyReLU(alpha=0.2))(mid_dense)
                        mid_dense = TimeDistributed(Dense(230, name='mid_dense_2'))(mid_dense) 
                        mid_dense = TimeDistributed(LeakyReLU(alpha=0.35))(mid_dense)
                        mid_dense = TimeDistributed(Dense(100, name='mid_dense_3'))(mid_dense) 
                        mid_dense = TimeDistributed(LeakyReLU(alpha=0.1))(mid_dense)
                        output = TimeDistributed(Dense(label_num, activation='softmax'))(mid_dense)
                    else:
                        activity_input = Input(shape=(max_activity_len, embedding_size), dtype='float32')
                        mask = Masking(mask_value=0)(activity_input)
                        forwards = LSTM(256, name='forwards_1', return_sequences=True)(mask)
                        forwards = Dropout(0.4)(forwards)
                        #forwards = LSTM(128, name='forwards_2', return_sequences=True)(forwards)
                        #forwards = Dropout(0.3)(forwards)
                        output = TimeDistributed(Dense(label_num, activation='softmax'))(forwards)

                    model = Model(input=activity_input, output=output)  
                    if label_num==2:
                        model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
                    else:
                        model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=[metrics.categorical_accuracy])
                    model.summary()

                    # get seq_emb output
                    seq_emb_output = seq_emb.predict(X_train_activity)
                    X_train_emb = np.zeros(shape=[len(train_col_size), max_activity_len, embedding_size], dtype='float32') 

                    for j, col_size in enumerate(train_col_size):
                        for p in range(col_size):
                            X_train_emb[j,p,:] = seq_emb_output[j,p,:]
                    del seq_emb_output 
                    gc.collect()

                    # train model
                    es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')
                    X_train_emb, X_valid_emb, y_train_activity, y_valid_activity = train_test_split(X_train_emb, y_train_activity, test_size=0.15, shuffle= True)
                    start = time.time()
                    hist = model.fit(X_train_emb, y_train_activity, batch_size=batch_size, 
                                     validation_data = (X_valid_emb, y_valid_activity), epochs=20, callbacks=[es])
                    end = time.time()
                    m5_runT.append(end-start)
                    print(" ********* seq%d-M5 Learning time: %dsec ********* " %(seq_std,int(end-start)))
                    train_result = hist.history
                    print('\nTrain_result\n')
                    print(train_result)
                    del X_train_emb, X_valid_emb, y_train_activity, y_valid_activity
                    gc.collect()

                    if not save_off:
                        # Save model
                        model_json = model.to_json()
                        model_name = data_name[path]+"_newM5_limit100_task1_day"+str(seq_std)+"_cv"+str(i)+".json"
                        weight_name = data_name[path]+"_newM5_limit100_task1_day"+str(seq_std)+"_cv"+str(i)+".h5"
                        with open(model_name,"w") as json_file :
                            json_file.write(model_json)
                            model.save_weights(weight_name)
                        print("Saved model to disk\n\n\n")  

                    #========================================================================================
                    # TEST DATA
                    #========================================================================================
                    if test_mode:
                        test_data = all_data[30:50]#i*splitLength:(i+1)*splitLength] 
                        test_history = all_history[30:50]#i*splitLength:(i+1)*splitLength]
                        test_3to1 = sum([len(r) for r in test_data])
                        test_time = all_time[train_3to1:train_3to1+test_3to1]#i*splitLength:(i+1)*splitLength]
                        #test_stream = all_stream[train_3to1 : train_3to1+test_3to1]
                        test_workday = all_workday[train_3to1:train_3to1+test_3to1]
                        test_recentday = all_recentday[train_3to1:train_3to1+test_3to1]
                        test_openday = all_openday[train_3to1:train_3to1+test_3to1]
                        test_activitycnt = all_activitycnt[train_3to1:train_3to1+test_3to1]
                        test_cc = all_cc[train_3to1:train_3to1+test_3to1]
                        test_writer = all_writer[train_3to1:train_3to1+test_3to1]
                        test_bugid = all_bugid[train_3to1:train_3to1+test_3to1]
                        test_total = all_total[train_3to1:train_3to1+test_3to1]
                    else:
                        test_data = all_data[i*splitLength:(i+1)*splitLength]
                        test_history = all_history[i*splitLength:(i+1)*splitLength]
                        test_3to1 = sum([len(r) for r in test_data])       
                        test_time = all_time[train_3to1 : train_3to1+test_3to1]#i*splitLength]
                        #test_stream = all_stream[train_3to1:train_3to1+test_3to1]
                        test_workday = all_workday[train_3to1:train_3to1+test_3to1]
                        test_recentday = all_recentday[train_3to1:train_3to1+test_3to1]
                        test_openday = all_openday[train_3to1:train_3to1+test_3to1]
                        test_activitycnt = all_activitycnt[train_3to1:train_3to1+test_3to1]
                        test_cc = all_cc[train_3to1:train_3to1+test_3to1]
                        test_writer = all_writer[train_3to1:train_3to1+test_3to1]
                        test_bugid = all_bugid[train_3to1:train_3to1+test_3to1]
                        test_total = all_total[train_3to1:train_3to1+test_3to1]
                    # ===================================================================== 
                    updated_test_data = []    
                    updated_test_history = []    
                    updated_test_time = []
                    #updated_test_stream = []
                    #updated_test_workday = []
                    #updated_test_bugid = []

                    j=0
                    for bug1, bug2 in zip(test_data, test_history):
                        test_data_list = []
                        test_history_list = []
                        if len(bug1)>=min_activity_len:
                            for act1, act2 in zip(bug1, bug2):
                                current_test_filter = [word for word in act1 if word in vocabulary]
                                his_current_test_filter = [word for word in act2 if word in vocabulary]
                                test_data_list.append(current_test_filter)
                                test_history_list.append(his_current_test_filter)
                                #updated_test_time.append(test_time[j])
                                #updated_test_stream.append(test_stream[j])
                                #updated_test_workday.append(test_workday[j])
                                #updated_test_bugid.append(test_bugid[j])
                                j+=1
                            updated_test_data.append(test_data_list)
                            updated_test_history.append(test_history_list)
                        else:
                            j+=len(bug1)

                    del test_data,  test_history, #test_bugid, test_workday, test_time
                    gc.collect()

                    # ===================================================================== 

                    #updated_test_time = [0 if x<=class_std else 1 for x in updated_test_time]

                    curr_split = total_split[data_name[path]][0]
                    updated_test_workday = [0 if x<=curr_split[0] else 1 if x<=curr_split[1] else 2 for x in test_workday]
                    updated_test_recentday = []
                    for x in test_recentday:
                        tmp = 0 
                        for j,y in enumerate(x):
                            tmp+= pow(2,j)*y
                        updated_test_recentday.append(tmp) 
                    curr_split = total_split[data_name[path]][1]
                    updated_test_openday = [0 if x<=curr_split[0] else 1 if x<=curr_split[1] else 2 if x<=curr_split[2] else 3 if x<=curr_split[3] else 4 for x in test_openday]
                    curr_split = total_split[data_name[path]][2]
                    updated_test_activitycnt = [0 if x<=curr_split[0] else 1 if x<=curr_split[1] else 2 if x<=curr_split[2] else 3 if x<=curr_split[3] else 4 for x in test_activitycnt]
                    curr_split = total_split[data_name[path]][3]
                    updated_test_cc = [0 if x<=curr_split[0] else 1 if x<=curr_split[1] else 2 if x<=curr_split[2] else 3 for x in test_cc]
                    updated_test_writer = test_writer
                    updated_test_bugid = test_bugid
                    
                    updated_test_data_1d = [y for x in updated_test_data for y in x]
                    updated_test_history_1d = [y for x in updated_test_history for y in x]
                    test_col_size = [len(x) for x in updated_test_data]
                    test_n = len(updated_test_data_1d)

                    test_df =pd.DataFrame({'x1': updated_test_data_1d, 'x2': updated_test_history_1d, 
                                            's1': updated_test_workday, 's2':updated_test_recentday, 's3':updated_test_activitycnt,
                                            's4':updated_test_cc, 's5':updated_test_writer, 's6':updated_test_openday,
                                            'bugid':updated_test_bugid,
                                            'y1': test_time})
                    del updated_test_data, updated_test_history, #test_workday, test_recentday,test_activitycnt
                    gc.collect()

                    # Task type
                    if task2_nextAct:
                        test_nextact = []
                        q=0
                        for col_size in test_col_size:
                            for x in range(col_size):
                                if x==col_size-1:
                                    test_nextact.append(-1)
                                else:
                                    test_nextact.append(test_time[q+x]-test_time[q+x+1])
                            q+=col_size
                        test_df['y1'] = test_nextact
                        test_df = test_df[test_df.y1!=-1]
                        test_n = len(test_df)
                        # new test_col_size
                        cnt=0
                        test_col_size = []
                        prev_id = test_df.iloc[0]['bugid']
                        for x in test_df.bugid.values:
                            if prev_id != x:
                                test_col_size.append(cnt)
                                cnt = 0
                            prev_id = x
                            cnt+=1
                        test_col_size.append(cnt)
                    elif task3_outlier or task4_lifetime:
                        test_df['y1'] = test_total

                    # ===================================================================== 
                    X_test_last = np.zeros(shape=[test_n, max_sentence_len, embed_size_word2vec], dtype='float32') # len(updated_test_data) # test_len*2
                    X_test_history = np.zeros(shape=[test_n, max_his_len, embed_size_word2vec], dtype='float32') # len(updated_test_history)
                    #X_test_stream = np.empty(shape=[test_n, limit_day], dtype='float32')
                    #X_test_workday = np.empty(shape=[test_n, max_workday], dtype='float32')
                    Y_test = np.empty(shape=[test_n,1], dtype='int32') # len(updated_test_time)

                    j=0
                    for curr_row1, curr_row2 in zip(test_df.x1.values, test_df.x2.values):
                        if len(curr_row1)>max_sentence_len:
                            start_loc = len(curr_row1) - max_sentence_len
                        else: 
                            start_loc = 0
                        sequence_cnt = 0    
                        for item1 in curr_row1[start_loc:]:
                            if item1 in vocabulary:
                                X_test_last[j, sequence_cnt, :] = wordvec_model[item1] 
                                sequence_cnt = sequence_cnt + 1                
                                if sequence_cnt == max_sentence_len-1:
                                    break  
                        for k in range(sequence_cnt, max_sentence_len):
                            X_test_last[j, k, :] = np.zeros((1,embed_size_word2vec))   
                        if len(curr_row2)>max_his_len:
                            start_loc = len(curr_row2) - max_his_len
                        else: 
                            start_loc = 0
                        sequence_cnt = 0
                        for item2 in curr_row2[start_loc:]:
                            if item2 in vocabulary:
                                X_test_history[j, sequence_cnt, :] = wordvec_model[item2] 
                                sequence_cnt = sequence_cnt + 1                
                                if sequence_cnt == max_his_len-1:
                                        break 
                        for k in range(sequence_cnt, max_his_len):
                            X_test_history[j, k, :] = np.zeros((1,embed_size_word2vec))
                        if test_df.y1.values[j]<=lamb:
                            Y_test[j,0] = 1
                        else:
                            Y_test[j,0] = 0
                        j+=1

                    # Statistic
                    X_test_workday = np.asarray(test_df.s1.values, dtype=np.float32)
                    class_num = len(total_split[data_name[path]][0])+1
                    X_test_workday = np_utils.to_categorical(X_test_workday, class_num)
                    X_test_openday = np.asarray(test_df.s6.values, dtype=np.float32)
                    class_num = len(total_split[data_name[path]][1])+1
                    X_test_openday = np_utils.to_categorical(X_test_openday, class_num)
                    X_test_recentday = np.asarray(test_df.s2.values, dtype=np.float32)
                    class_num = 8
                    X_test_recentday = np_utils.to_categorical(X_test_recentday, class_num)
                    X_test_activitycnt = np.asarray(test_df.s3.values, dtype=np.float32)
                    class_num = len(total_split[data_name[path]][2])+1
                    X_test_activitycnt = np_utils.to_categorical(X_test_activitycnt, class_num)
                    X_test_cc = np.asarray(test_df.s4.values, dtype=np.float32)
                    class_num = len(total_split[data_name[path]][3])+1
                    X_test_cc = np_utils.to_categorical(X_test_cc, class_num)
                    X_test_writer = np.array([np.array(x, dtype=np.float32) for x in test_df.s5.values])

                    X_test_stat = np.concatenate((X_test_workday, X_test_recentday),axis=1)
                    X_test_stat = np.concatenate((X_test_stat, X_test_openday),axis=1)
                    X_test_stat = np.concatenate((X_test_stat, X_test_activitycnt),axis=1)
                    X_test_stat = np.concatenate((X_test_stat, X_test_cc),axis=1)
                    X_test_stat = np.concatenate((X_test_stat, X_test_writer),axis=1)

                    # ===================================================================== 
                    embedding_test_output = model_1.predict([X_test_last, X_test_history, X_test_stat])
                    del model_1
                    X_test_activity = np.zeros(shape=[len(test_col_size), max_activity_len, embedding_size], dtype='float32') 
                    y_test_activity = np.full((len(test_col_size), max_activity_len, 1), -1, dtype='int32') 
                    act_idx = 0 # 200d_activity_output row
                    for j, col_size in enumerate(test_col_size):
                        for p in range(col_size):
                            X_test_activity[j,p,:] = embedding_test_output[act_idx+p]
                            if task3_outlier:
                                if test_df.y1.values[act_idx+p]>=seq_std:
                                    y_test_activity[j,p,:] = 1
                                else:
                                    y_test_activity[j,p,:] = 0
                            elif task4_lifetime or task5_multiFixtime:
                                y_test_activity[j,p,:] = test_df.y1.values[act_idx+p]
                            else:
                                if test_df.y1.values[act_idx+p]<=seq_std:
                                    y_test_activity[j,p,:] = 1
                                else:
                                    y_test_activity[j,p,:] = 0
                        act_idx += col_size

                    # 
                    seq_emb_output = seq_emb.predict(X_test_activity)
                    X_test_emb = np.zeros(shape=[len(test_col_size), max_activity_len, embedding_size], dtype='float32') 
                    act_idx = 0 # 200d_activity_output row

                    for j, col_size in enumerate(test_col_size):
                        for p in range(col_size):
                            X_test_emb[j,p,:] = seq_emb_output[j,p,:]
                        act_idx += col_size
                    del seq_emb_output 
                    gc.collect()

                    X_test_activity = X_test_emb
                    '''
                    #========================================================================================
                    # M3 PREDICT & ACCURACY 
                    #========================================================================================
                    if mode==0:
                        print('\n\n******************* M3 - Embedding RNN *******************') 
                    elif mode==1:
                        print('\n\n******************* M4 - Embedding RNN *******************') 
                    if doc2vec_rnn:
                        predict = m3.predict([X_test_last, X_test_history, X_test_stat])  
                        del X_test_last, X_test_history, X_test_stat
                    elif doc2vec_avg:
                        predict = m3.predict([X_test_avg, X_test_stat]) 
                        del X_test_avg, X_test_stat
                    predictY = np.argmax(predict, axis=1)
                    corrects = np.nonzero(predictY.reshape((-1,1)) == Y_test) #.reshape((-1,))
                    accu = len(corrects[0])/len(Y_test)
                    print("Total test accuracy : %.2f" % (accu*100))
                    print('\nConfusion Matrix')
                    cm = confusion_matrix(Y_test, predictY)
                    print(cm)
                    total = sum(cm[0])+sum(cm[1])
                    #print('Predict 0: %.2f' % ((cm[0,0]+cm[1,0])/total*100))
                    #print('Predict 1: %.2f' % ((cm[0,1]+cm[1,1])/total*100))
                    f1 = (f1_score(Y_test, predictY, average="micro"))
                    prec= (precision_score(Y_test, predictY, average="micro"))
                    recall = (recall_score(Y_test, predictY, average="micro"))
                    print('\nf1_score: %.2f' % (f1*100))
                    print('precision_score: %.2f'% (prec*100)) # class 0 accuracy
                    print('recall_score: %.2f' % (recall*100)) # class 1 accuracy
                    print('\n%.2f %.2f %.2f %.2f\n' %(prec*100, recall*100, f1*100, accu*100))
                    print()
                    del m3
                    gc.collect()
                    '''
                    #========================================================================================
                    # PREDICT & ACCURACY
                    #========================================================================================
                    if task4_lifetime or task5_multiFixtime:          
                        y_test_activity = y_test_activity.reshape((-1,))
                        padding_idx = np.where(y_test_activity==-1)
                        y_test_activity = np.delete(y_test_activity, padding_idx)
                        y_test_activity_topk = np_utils.to_categorical(y_test_activity, label_num)
                        
                        predict = model.predict(X_test_activity)
                        predictY = np.argmax(predict, axis=2)
                        predict = predict.reshape((-1,seq_std))
                        predict = np.delete(predict, padding_idx, 0)
                       
                        predictY = predictY.reshape((-1,))
                        predictY = np.delete(predictY, padding_idx) 
                        
                        print()
                        for k_ in [1,5,10,50]:
                            r = K.eval(metrics.top_k_categorical_accuracy(y_test_activity_topk, predict, k=k_))
                            print('Top',k_,'Accuracy: %.2f' % (r*100))
                        print('\nConfusion Matrix')
                        cm = confusion_matrix(y_test_activity, predictY)
                        print(cm)
                        print('\nf1_score: %.2f' % (f1_score(y_test_activity, predictY, average="macro")*100))
                        print('precision_score: %.2f'% (precision_score(y_test_activity, predictY, average="macro")*100)) # class 0 accuracy
                        print('recall_score: %.2f' % (recall_score(y_test_activity, predictY, average="macro")*100)) # class 1 accuracy
                        print()
                    else:
                        print('\n\n******************* M5 -sequence RNN *******************')  
                        y_test_activity = y_test_activity.reshape((-1,))
                        padding_idx = np.where(y_test_activity==-1)
                        y_test_activity = np.delete(y_test_activity, padding_idx)
                        print('len(y_test_activity)', len(y_test_activity))
                        predict = model.predict(X_test_activity)
                        predictY = np.argmax(predict, axis=2)
                        print('predictY',predictY[:30])
                        predictY = predictY.reshape((-1,))
                        predictY = np.delete(predictY, padding_idx)
                        print('len(predictY)', len(predictY))
                        corrects = np.nonzero(predictY == y_test_activity) #.reshape((-1,))
                        accu = len(corrects[0])/len(y_test_activity)
                        print("Total test accuracy : %.2f" % (accu*100))
                        print('\nConfusion Matrix')
                        cm = confusion_matrix(y_test_activity, predictY)
                        print(cm)
                        f1 = f1_score(y_test_activity, predictY, average="binary")
                        prec = precision_score(y_test_activity, predictY, average="binary")
                        recall = recall_score(y_test_activity, predictY, average="binary")
                        print('\nf1_score: %.2f' % (f1*100))
                        print('precision_score: %.2f'% (prec*100)) # class 0 accuracy
                        print('recall_score: %.2f' % (recall*100)) # class 1 accuracy
                        total = sum(cm[0])+sum(cm[1])
                        print('Predict 0: %.2f' % ((cm[0,0]+cm[1,0])/total*100))
                        print('Predict 1: %.2f' % ((cm[0,1]+cm[1,1])/total*100))
                        print('\n%.2f %.2f %.2f %.2f\n' %(prec*100, recall*100, f1*100, accu*100))
                        print()
                    del model

            #print('\n******************',emb_std, 'M3_runT, average :',m3_runT, sum(m3_runT)/len(m3_runT),'******************')
            print('******************',seq_std, 'M5_runT, average :', m5_runT, sum(m5_runT)/len(m5_runT),'******************\n\n')

============= d_size ============= 1
class 1 비율 36.31463685363146
Total length:  30428
emb_std: 7
seq_std: 5
train_balancing False
CV 6


/home/yksi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:258: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/yksi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:271: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/yksi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:382: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("ou...)`
/home/yksi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:434: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("mi...)`




******************* M3-RNN *******************


******************* M5- seqence RNN *******************


/home/yksi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:475: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ba...)`
/home/yksi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:529: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ti...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_39 (InputLayer)        (None, 10, 200)           0         
_________________________________________________________________
masking_29 (Masking)         (None, 10, 200)           0         
_________________________________________________________________
forwards_1 (LSTM)            (None, 10, 256)           467968    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 256)           0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 10, 2)             514       
Total params: 468,482
Trainable params: 468,482
Non-trainable params: 0
_________________________________________________________________
Train on 14106 samples, validate on 2490 samples
Epoch 1/20
14106/14106 [==============================] - 24s 2ms/step - loss: 0.4352 -

...



******************* M3-RNN *******************


******************* M5- seqence RNN *******************


/home/yksi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:475: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ba...)`
/home/yksi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:529: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ti...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_159 (InputLayer)       (None, 10, 200)           0         
_________________________________________________________________
masking_125 (Masking)        (None, 10, 200)           0         
_________________________________________________________________
forwards_1 (LSTM)            (None, 10, 256)           467968    
_________________________________________________________________
dropout_25 (Dropout)         (None, 10, 256)           0         
_________________________________________________________________
time_distributed_31 (TimeDis (None, 10, 2)             514       
Total params: 468,482
Trainable params: 468,482
Non-trainable params: 0
_________________________________________________________________
Train on 23511 samples, validate on 4149 samples
Epoch 1/20
23511/23511 [==============================] - 57s 2ms/step - loss: 0.4177 -

/home/yksi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:711: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/yksi/.local/lib/python3.6/site-packages/ipykernel_launcher.py:724: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).




******************* M5 -sequence RNN *******************
len(y_test_activity) 9173
predictY [[0 0 0 0 1 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1]
 [0 0 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [0 0 0 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1]
 [0 0 0 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1]
 [0 1 1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1 1]
 [0 0 0 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1 1]
 [0 0 0 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]
len(predictY) 9173
Total test accuracy : 73.06

Confusion Matrix
[[4460 1307]
 [1164 2242]]

f1_score: 64.47
precision_score: 63.17
recall_score: 65.83
Predict 0: 61.31
Predict 1: 38.69

63.17 65.83 64.47 73.06


**